In [18]:
# 参考Bert WordPiece的逻辑，local实现tokenization功能

In [1]:
file_path = """WordPiece_local/bert-large-uncased-vocab.txt"""

In [2]:
vocab = []

In [3]:
with open(file_path,"r") as f:
    for line in f.readlines():
        # print(line.strip())
        vocab.append(line.strip())

In [7]:
vocab[10009]

'interpreted'

In [16]:
# from bert import tokenization

In [47]:
import collections

temp = collections.OrderedDict()

In [50]:
temp["AAA"]=2
temp["BBB"]=1

In [51]:
temp

OrderedDict([('AAA', 2), ('BBB', 1)])

In [1]:
!pip install -q -U tensorflow-text

In [2]:
import tensorflow as tf
import tensorflow_text as text
import functools

In [3]:
examples = {
    "text_a": [
      b"Sponge bob Squarepants is an Avenger",
      b"Marvel Avengers"
    ],
    "text_b": [
     b"Barack Obama is the President.",
     b"President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))

2021-12-27 11:25:46.845022: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Sponge bob Squarepants is an Avenger'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'Barack Obama is the President.'>}

In [4]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5
 
_VOCAB_SIZE = len(_VOCAB)

In [5]:
lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=_VOCAB,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
      value_dtype=tf.int64),
      num_oov_buckets=1
)

In [6]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.string)
bert_tokenizer.tokenize(examples["text_a"])

<tf.RaggedTensor [[[b'Sp', b'##onge'], [b'bob'], [b'Sq', b'##uare', b'##pants'], [b'is'], [b'an'], [b'A', b'##ven', b'##ger']], [[b'Mar', b'##vel'], [b'A', b'##ven', b'##gers']]]>

In [7]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.int64)
segment_a = bert_tokenizer.tokenize(examples["text_a"])
segment_a

<tf.RaggedTensor [[[22, 9], [24], [23, 11, 10], [28], [14], [15, 13, 7]], [[18, 12], [15, 13, 8]]]>

In [8]:
vocab_lookup_table = """WordPiece_local/bert-large-uncased-vocab.txt"""
# suffix_indicator='##', 
# max_bytes_per_word=100,
# max_chars_per_token=None, 
# token_out_type=dtypes.int64,
# unknown_token='[UNK]', 
# split_unknown_characters=False,
# lower_case=False, 
# keep_whitespace=False, 
# normalization_form=None,
# preserve_unused_token=False, 
# basic_tokenizer_class=BasicTokenizer

In [10]:
import dtypes

ModuleNotFoundError: No module named 'dtypes'

In [12]:
test = text.BertTokenizer(
    vocab_lookup_table, suffix_indicator='##', max_bytes_per_word=100,
    max_chars_per_token=None, 
    # token_out_type=dtypes.int64,
    unknown_token='[UNK]', split_unknown_characters=False,
    lower_case=False, keep_whitespace=False, normalization_form=None,
    preserve_unused_token=False, 
    # basic_tokenizer_class=BasicTokenizer
)

In [15]:
test.tokenize("hello world")

<tf.RaggedTensor [[[7592], [2088]]]>

In [18]:
test.detokenize(test.tokenize("hello world, , ,;lkj;lkj;lkjiuginunnj，  "))

<tf.RaggedTensor [[[b'hello'], [b'world'], [b','], [b','], [b','], [b';'], [b'lkj'], [b';'], [b'lkj'], [b';'], [b'lkjiuginunnj'], [b'\xef\xbc\x8c']]]>